In [54]:
import pandas as pd
import string
# FRE SCORE
from textstat import flesch_reading_ease
# SENTIMENT ANALYSIS USING VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#POS TAGGING
import spacy
#VECTORISING TEXT AND CREATING PIPELINE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
#COSINE SIMILARITY BETWEEN REVIEWS
from sklearn.metrics.pairwise import cosine_similarity

import time
import pandas as pd
import numpy as np

nlp = spacy.load('en_core_web_sm')

In [1]:
#IMPORT CLASSIFICATION MODULES
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("Datasets/Yelp Dataset Reduced.csv")

In [3]:
len(df)

20000

In [4]:
df.columns

Index(['Unnamed: 0', 'USER_ID', 'PRODUCT_ID', 'RATING', 'DATE', 'LABEL',
       'REVIEW_TEXT', 'AVERAGE_RATING', 'RATING_DEVIATION',
       'TOTAL_PRODUCT_REVIEWS', 'REVIEW_LENGTH', 'RATING_CATEGORY',
       'SINGLE_RATING_CATEGORY', 'REVIEW_COUNT_DATE',
       'SAME_DATE_MULTIPLE_REVIEWS', 'MAX_USER_REVIEWS_DAY',
       'TIMESTAMP_DIFFERENCE', 'AVERAGE_USER_REVIEW_LENGTH',
       'TOTAL_USER_REVIEWS', 'PERCENTAGE_POSITIVE_REVIEWS',
       'RATIO_POSITIVE_NEGATIVE'],
      dtype='object')

In [9]:
for col in df.columns:
    print(col + "\n")

Unnamed: 0

USER_ID

PRODUCT_ID

RATING

DATE

LABEL

REVIEW_TEXT

AVERAGE_RATING

RATING_DEVIATION

TOTAL_PRODUCT_REVIEWS

REVIEW_LENGTH

RATING_CATEGORY

SINGLE_RATING_CATEGORY

REVIEW_COUNT_DATE

SAME_DATE_MULTIPLE_REVIEWS

MAX_USER_REVIEWS_DAY

TIMESTAMP_DIFFERENCE

AVERAGE_USER_REVIEW_LENGTH

TOTAL_USER_REVIEWS

PERCENTAGE_POSITIVE_REVIEWS

RATIO_POSITIVE_NEGATIVE



In [33]:
#ADD POS TAGS
def add_pos_tags(df):
    def count_pos(Pos_counts, pos_type):
        pos_count = Pos_counts.get(pos_type, 0)
        return pos_count

    def pos_counts(text):
        doc = nlp(text)
        Pos_counts = doc.count_by(spacy.attrs.POS)
        return Pos_counts

    poscounts = df['REVIEW_TEXT'].apply(pos_counts)
    df['NUM_NOUNS'] = poscounts.apply(lambda counts: count_pos(counts, spacy.parts_of_speech.NOUN))
    df['NUM_VERBS'] = poscounts.apply(lambda counts: count_pos(counts, spacy.parts_of_speech.VERB))
    df['NUM_ADJECTIVES'] = poscounts.apply(lambda counts: count_pos(counts, spacy.parts_of_speech.ADJ))
    df['NUM_ADVERBS'] = poscounts.apply(lambda counts: count_pos(counts, spacy.parts_of_speech.ADV))


In [8]:
#ADD NAMED ENTITIES
def add_named_entities(df):
    def count_entities(text):
        doc = nlp(text)
        ent_count = len([ent.text for ent in doc.ents])
        return ent_count

    df['NUM_NAMED_ENTITIES'] = df['REVIEW_TEXT'].apply(count_entities)


In [12]:
#ADD SENTIMENT SCORE
def add_vader_sentiment_score(df):
    sid = SentimentIntensityAnalyzer()

    df['SENTIMENT_SCORE'] = df['REVIEW_TEXT'].apply(
        lambda d: sid.polarity_scores(d)['compound'])


In [10]:
#ADD READABILITY SCORE
def add_readability_score(df):
    df['READABILITY_FRE'] = df['REVIEW_TEXT'].apply(
        lambda d: flesch_reading_ease(d))


In [ ]:
add_pos_tags(df)

In [46]:
df.head()

,Unnamed: 0,USER_ID,PRODUCT_ID,RATING,DATE,LABEL,REVIEW_TEXT,AVERAGE_RATING,RATING_DEVIATION,TOTAL_PRODUCT_REVIEWS,...,PERCENTAGE_POSITIVE_REVIEWS,RATIO_POSITIVE_NEGATIVE,READABILITY_FRE,NUM_NAMED_ENTITIES,SENTIMENT_SCORE,NUM_NOUNS,NUM_VERBS,NUM_ADJECTIVES,NUM_ADVERBS,PREPROC_REVIEW_TEXT
0,144828,66563,416,4,2014-12-10,-1,Great.....,3.767293,0.232707,2183,...,100.0,6.157377,121.22,0,0.0000,"{95: 12, 92: 30, 89: 7, 100: 11, 86: 5, 85: 13...","{84: 9, 92: 9, 97: 9, 89: 3, 90: 5, 95: 3, 100...","{95: 12, 100: 11, 93: 2, 92: 18, 86: 12, 89: 3...","{87: 5, 86: 6, 90: 6, 84: 8, 92: 17, 85: 5, 97...",great
1,157607,74755,449,4,2013-03-26,1,My family and I had Bubby's brunch on a Saturd...,3.396552,0.603448,812,...,100.0,3.121827,82.65,9,0.9162,"{95: 12, 92: 30, 89: 7, 100: 11, 86: 5, 85: 13...","{84: 9, 92: 9, 97: 9, 89: 3, 90: 5, 95: 3, 100...","{95: 12, 100: 11, 93: 2, 92: 18, 86: 12, 89: 3...","{87: 5, 86: 6, 90: 6, 84: 8, 92: 17, 85: 5, 97...",family Bubby brunch Saturday morning Â get wai...
2,70401,49165,237,3,2011-10-11,1,"I really like this place, but they need to get...",3.799003,0.799003,602,...,100.0,6.082353,67.25,1,-0.7681,"{95: 12, 92: 30, 89: 7, 100: 11, 86: 5, 85: 13...","{84: 9, 92: 9, 97: 9, 89: 3, 90: 5, 95: 3, 100...","{95: 12, 100: 11, 93: 2, 92: 18, 86: 12, 89: 3...","{87: 5, 86: 6, 90: 6, 84: 8, 92: 17, 85: 5, 97...",like place need menu sort finally go brunch ea...
3,124810,75653,363,5,2014-01-14,1,This is one of my favorite places in the US. A...,3.990361,1.009639,2075,...,100.0,9.121951,84.17,2,0.9687,"{95: 12, 92: 30, 89: 7, 100: 11, 86: 5, 85: 13...","{84: 9, 92: 9, 97: 9, 89: 3, 90: 5, 95: 3, 100...","{95: 12, 100: 11, 93: 2, 92: 18, 86: 12, 89: 3...","{87: 5, 86: 6, 90: 6, 84: 8, 92: 17, 85: 5, 97...",favorite place awesome reservation place pack ...
4,42068,32402,100,4,2014-12-02,1,Make sure you go with a small group of friends...,3.951812,0.048188,2677,...,100.0,8.734545,81.67,2,0.8805,"{95: 12, 92: 30, 89: 7, 100: 11, 86: 5, 85: 13...","{84: 9, 92: 9, 97: 9, 89: 3, 90: 5, 95: 3, 100...","{95: 12, 100: 11, 93: 2, 92: 18, 86: 12, 89: 3...","{87: 5, 86: 6, 90: 6, 84: 8, 92: 17, 85: 5, 97...",sure small group friend like share food try ta...


In [13]:
add_readability_score(df)
add_named_entities(df)
add_vader_sentiment_score(df)
add_pos_tags(df)

In [14]:
#PREPROCESS REVIEW TEXT
from spacy.lang.en.stop_words import STOP_WORDS as stop_words

def preprocess_text(text):

    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_stop]
    lemmas = [lemma for lemma in lemmas if lemma.isalpha()]
    preprocessed_text = ' '.join(lemmas)

    return preprocessed_text


In [16]:
#PREPROCESS THE REVIEW TEXT
df['PREPROC_REVIEW_TEXT'] = df['REVIEW_TEXT'].apply(preprocess_text)

In [39]:
#CONVERT TIMESTAMP DIFFERENCE TO INTEGER
df['TIMESTAMP_DIFFERENCE'] = df['TIMESTAMP_DIFFERENCE'].str.replace(' days', '').astype(int)

In [56]:
#RE-ENCODE LABEL
df['LABEL'] = df['LABEL'].apply(lambda x: 0 if x<0 else 1)

In [40]:
df.to_csv("Datasets/Yelp_Reviews_Labelled.csv")

In [2]:
# DICTIONARY WITH NAME AND COMMAND TO INSTANTIATE DIFFERENT MODELS
classifiers = {}
classifiers.update({"XGBClassifier": XGBClassifier(eval_metric='logloss',objective='binary:logistic',use_label_encoder=False)})
classifiers.update({"CatBoostClassifier": CatBoostClassifier(silent=True)})
classifiers.update({"LinearSVC": LinearSVC(max_iter=10000)})
classifiers.update({"MultinomialNB": MultinomialNB()})
classifiers.update({"LGBMClassifier": LGBMClassifier()})
classifiers.update({"RandomForestClassifier": RandomForestClassifier()})
classifiers.update({"DecisionTreeClassifier": DecisionTreeClassifier()})
classifiers.update({"ExtraTreeClassifier": ExtraTreeClassifier()})
classifiers.update({"AdaBoostClassifier": AdaBoostClassifier()})
classifiers.update({"KNeighborsClassifier": KNeighborsClassifier()})
classifiers.update({"RidgeClassifier": RidgeClassifier()})
classifiers.update({"SGDClassifier": SGDClassifier()})
classifiers.update({"BaggingClassifier": BaggingClassifier()})
classifiers.update({"BernoulliNB": BernoulliNB()})
classifiers.update({"LogisticRegression": LogisticRegression()})
classifiers.update({"SVM": SVC()})

/Users/kpandey/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [57]:
features_text = df['PREPROC_REVIEW_TEXT']
features_numeric = df[[
    'SENTIMENT_SCORE', 'READABILITY_FRE', 'NUM_NAMED_ENTITIES',
    'RATING',
    'AVERAGE_RATING', 'RATING_DEVIATION',
       'TOTAL_PRODUCT_REVIEWS', 'REVIEW_LENGTH', 'RATING_CATEGORY',
       'SINGLE_RATING_CATEGORY', 'REVIEW_COUNT_DATE',
       'SAME_DATE_MULTIPLE_REVIEWS', 'MAX_USER_REVIEWS_DAY',
       'TIMESTAMP_DIFFERENCE', 'AVERAGE_USER_REVIEW_LENGTH',
       'TOTAL_USER_REVIEWS', 'PERCENTAGE_POSITIVE_REVIEWS',
       'RATIO_POSITIVE_NEGATIVE'
]]

In [60]:
labels = df['LABEL']

In [48]:
df.fillna(value=0, inplace=True)

In [61]:
# TRAIN-TEST SPLIT
X_text_train, X_text_test, X_numeric_train, X_numeric_test, y_train, y_test = train_test_split(
    features_text, features_numeric, labels, test_size=0.2, random_state=42
)

# VEFCTORISE THE TEXTUAL FEATURES
tfidf = TfidfVectorizer()
X_text_train_tfidf = tfidf.fit_transform(X_text_train)
X_text_test_tfidf = tfidf.transform(X_text_test)

# Standardize the numeric features
# scaler = StandardScaler()
# X_numeric_train = scaler.fit_transform(X_numeric_train)
# X_numeric_test = scaler.transform(X_numeric_test)

# Concatenate the TF-IDF matrix and numeric features
X_train = np.hstack((X_text_train_tfidf.toarray(), X_numeric_train))
X_test = np.hstack((X_text_test_tfidf.toarray(), X_numeric_test))


In [65]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')  # You can choose a different imputation strategy if needed
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Handle infinite values
X_train = np.nan_to_num(X_train, nan=0, posinf=1e9, neginf=-1e9)
X_test = np.nan_to_num(X_test, nan=0, posinf=1e9, neginf=-1e9)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [62]:
# CREATE A DATAFRAME OF MODELS WITH RUN TIME AND AUC SCORES
df_models = pd.DataFrame(
    columns=['model', 'run_time', 'accuracy', 'precision', 'f1_score'])

print("TRAINING DATA AND TESTING DATA CREATED")
i = 1

for key in classifiers:
    # STARTING TIME
    start_time = time.time()
    # CURRENT CLASSIFIER
    clf = classifiers[key]
    # TRAIN CLASSIFIER ON TRAINING DATA
    clf.fit(X_train, y_train)
    # MAKE PREDICTIONS USING CURRENT CLASSIFIER
    predictions = clf.predict(X_test)
    # CALCULATE ACCURACY
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    f1score = f1_score(y_test, predictions)

    row = {'model': key,
           'run_time': format(round((time.time() - start_time)/60, 2)),
           'accuracy': accuracy,
           'precision': precision,
           'f1_score': f1score
           }

    df_models = df_models._append(row, ignore_index=True)
    print(f"{i} MODEL DONE")
    i+=1;

df_models = df_models.sort_values(by='accuracy', ascending=False)

TRAINING DATA AND TESTING DATA CREATED


XGBoostError: [17:45:07] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/data/data.cc:1104: Check failed: valid: Input data contains `inf` or `nan`
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000014dd4e6d5 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000014ddd7440 unsigned long long xgboost::SparsePage::Push<xgboost::data::ArrayAdapterBatch>(xgboost::data::ArrayAdapterBatch const&, float, int) + 1280
  [bt] (2) 3   libxgboost.dylib                    0x000000014de2481b xgboost::data::SimpleDMatrix::SimpleDMatrix<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int) + 299
  [bt] (3) 4   libxgboost.dylib                    0x000000014ddd5985 xgboost::DMatrix* xgboost::DMatrix::Create<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char